In [12]:
import requests 
from bs4 import BeautifulSoup
import re
import pandas as pd
from time import sleep

In [13]:
headers = {'User-Agent': 'Mozilla/5.0'}                    # задаем header для загрузки страницы
data=[]                                                    # создаём пустой список, в который будут собираться объявления

for i in range(1, 2):                                     # задаем цикл с номерами страниц для загрузки
    print(i)                                               # выводим на печать номер обрабатываемой страницы
    
    url = f'https://www.list.am/ru/category/56/{i}?type=1&n=1&_a39=2&_a3_1=60&_a3_2=100' # задаем url
    r = requests.get(url, headers=headers)                   # загружаем в переменную данные полученные с сайта
    sleep(3)                                                 # Задаем время ожидания между запросами на сайт
    soup = BeautifulSoup(r.text, "lxml")                     # Преобразуем дынные полученные requests для дальнейшей работы
    flats = soup.findAll('div', class_='dl')[1].find('div', class_='gl').findAll('a') # находим все объявления на странице
    
    
    for flat in flats:                                      # проходим по списку с объявлениями и извлекаем информацию о каждом
        
        try:
            link = 'https://www.list.am' + flat.get('href')  # извлекаем ссылку на объявление
            price = flat.find('div', class_='p').text        # извлекаем цену на квартиру
            information = flat.find('div', class_='at').text # извлекаем короткое описание квартиры

            result = re.findall(r'\d{1,3}', information) # Создаем промежуточную переменную с площадью и кол-ом комнат квартиры

            number = re.findall(r'\d+', link)[0]              # извлекаем номер объявления
            region = re.findall(r'\w+,', information)[0][:-1] # извлекаем регион
            rooms = int(result[0])                            # извлекаем количество комнат
            square = int(result[1])                           # извлекаем площадь
            value = re.findall(r'[\֏\$]', price)[0]           # извлекаем валюту цены
            cost = int(re.findall(r'\d+[,]?\d+[,]?\d+', price)[0].replace(',','')) # извлекаем цену
            if value == '$':                                  # если цена в долларах, конвертируем её в драммы
                cost *= 390
                value = '֏'

            data.append([number, region, cost, value, rooms, square, price+', ' +information, link]) # добавляем всю информацию

        except:
            print("Исключение было обработано") 

print('THE END')

1
THE END


In [14]:
headers = ['number', 'region', 'cost', 'value', 'rooms', 'square', 'information', 'link'] # задаем названия колонок
df = pd.DataFrame(data, columns=headers)                                                  # создаем DataFrame
df


,number,region,cost,value,rooms,square,information,link
0,20952847,Арабкир,585000,֏,2,92,"$1,500 в месяц, Арабкир, 2 ком., 92 кв.м., 11/...",https://www.list.am/ru/item/20952847
1,20958095,Канакер,400000,֏,3,98,"400,000 ֏ в месяц, Зейтун Канакер, 3 ком., 98 ...",https://www.list.am/ru/item/20958095
2,20974626,Канакер,270000,֏,2,73,"270,000 ֏ в месяц, Зейтун Канакер, 2 ком., 73 ...",https://www.list.am/ru/item/20974626
3,17083650,Кентрон,585000,֏,2,85,"$1,500 в месяц, Кентрон, 2 ком., 85 кв.м., 17/...",https://www.list.am/ru/item/17083650
4,19226332,Кентрон,507000,֏,2,65,"$1,300 в месяц, Кентрон, 2 ком., 65 кв.м., 12/...",https://www.list.am/ru/item/19226332
...,...,...,...,...,...,...,...,...
91,18717660,Давташен,300000,֏,3,80,"300,000 ֏ в месяц, Давташен, 3 ком., 80 кв.м.,...",https://www.list.am/ru/item/18717660
92,20807099,Кентрон,468000,֏,3,80,"$1,200 в месяц, Кентрон, 3 ком., 80 кв.м., 6/1...",https://www.list.am/ru/item/20807099
93,19979786,Кентрон,312000,֏,3,80,"$800 в месяц, Кентрон, 3 ком., 80 кв.м., 11/12...",https://www.list.am/ru/item/19979786
94,20567369,Канакер,380000,֏,3,100,"380,000 ֏ в месяц, Зейтун Канакер, 3 ком., 100...",https://www.list.am/ru/item/20567369


In [15]:
df.to_csv(r'D:\Ela\flats_Yerevan\flats_yerevan.csv', index=False, sep=';', encoding='utf8') # сохраняем в файл